# NN with pytorch
since I have two 1070 gpus, I'm considering to create a NN model with pyTorch
training on all data 


In [1]:
import pandas as pd
import numpy as np
import random
import pickle
X_train = pd.read_csv("data/train")

/home/stream/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
X_train_trial = X_train[['banner_pos','C1','device_type','device_conn_type','C14','C15','C16','C17','C18',\
                         'C19','C20','C21']].values

In [3]:
y_target = X_train.click.values

In [14]:
X_train_trial.shape

(40428967, 12)

In [17]:
import gc
from sklearn.feature_extraction import FeatureHasher
hasher = FeatureHasher(n_features=10,
            non_negative=True,
            input_type='string')

# create hashing features
## site
print("##---------hashing site----------##")

tmp_features = hasher.transform(X_train.site_id.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))
gc.collect()

tmp_features = hasher.transform(X_train.site_domain.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))
gc.collect()
    
tmp_features = hasher.transform(X_train.site_category.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))

gc.collect()

## app
print("##---------hashing app----------##")
tmp_features = hasher.transform(X_train.app_id.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))
gc.collect()

tmp_features = hasher.transform(X_train.app_domain.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))
gc.collect()
tmp_features = hasher.transform(X_train.app_category.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))

gc.collect()

## devices
print("##---------hashing devices----------##")
tmp_features = hasher.transform(X_train.device_id.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))
gc.collect()

tmp_features = hasher.transform(X_train.device_ip.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))
gc.collect()
    
tmp_features = hasher.transform(X_train.device_model.tolist()).toarray()
X_train_trial = np.column_stack((X_train_trial,tmp_features))
gc.collect()

##---------hashing app----------##
##---------hashing devices----------##


0

In [19]:
X_train=[]
import gc
gc.collect()

0

In [20]:
print(type(X_train_trial))
X_train_trial.shape

<type 'numpy.ndarray'>


(40428967, 102)

In [9]:
print(type(y_target))
y_target.shape

<type 'numpy.ndarray'>


(40428967,)

In [21]:
np.save('X_train.npy',X_train_trial)
np.save('y_target.npy',y_target)

# [Sec1 ] Create pytorch Data_loader

In [1]:
# create tensor
import numpy as np
X_train_trial = np.load('X_train.npy')
y_target = np.load('y_target.npy').astype(int)
print(X_train_trial.shape)
print(y_target.shape)


(40428967, 102)
(40428967,)


In [2]:
msk = np.random.rand(len(X_train_trial)) < 0.8

In [3]:
import torch
train_data = torch.from_numpy(X_train_trial[msk]).float()
train_labels = torch.from_numpy(y_target[msk])

In [4]:
test_data = torch.from_numpy(X_train_trial[~msk]).float()
test_labels = torch.from_numpy(y_target[~msk])

In [5]:
msk=None
import gc
gc.collect()

60

In [6]:
# create our dataloader first 
import torch.utils.data as data_utils

train = data_utils.TensorDataset(train_data, train_labels)
train_loader = data_utils.DataLoader(train, batch_size=30000, shuffle=True,num_workers=8)

test = data_utils.TensorDataset(test_data, test_labels)
test_loader = data_utils.DataLoader(test, batch_size=30000, shuffle=False,num_workers=8)

In [7]:
# Neural Networks
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(102, 51)
        #self.fc2 = nn.Linear(51, 25)
        self.fc3 = nn.Linear(51, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
import torch.optim as optim

net = Net()
net = nn.DataParallel(net.cuda())
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(net.parameters(), lr=1e-5)
optimizer = optim.SGD(net.parameters(), lr=1e-6, momentum=0.9,weight_decay=1e-2)

In [12]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
    count=0
    for i, data in enumerate(test_loader, 0):
        count+=1
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels.cuda())

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        # print statistics
        running_loss += loss.data[0]
    print('[%d, %5d] testing loss: %.3f' %(epoch + 1, count, running_loss / count))
    running_loss = 0.0
print('Finished Training')

KeyboardInterrupt: 

In [ ]:
# using all mean value in training set get loss: 0.45557787243041559
# didn't improve at all. 

In [ ]:
# model saving